# ANÁLISIS DE TENDENCIAS TEMPORALES

__Variables:__ FECH, FOBDOL

1. __FECH__ - Fecha del proceso (AÑO, MES)
2. __FOBDOL__ - Valor FOB en dólares

>"El valor FOB en dólares muestra cuánto valen las exportaciones al momento de salir del país. Este monto incluye todos los gastos necesarios para llevar la mercancía hasta el puerto y cargarla al barco o avión, como la producción, el empaque y el transporte interno. Sin embargo, no toma en cuenta el costo del seguro ni el transporte internacional hasta el destino final. Por eso, el FOB sirve para comparar el valor de los productos exportados de forma estándar y consistente entre diferentes períodos y destinos."

In [30]:
import sys
sys.path.append('..')
from utils.paths import get_file_path

from scipy import stats
import polars as pl
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import calendar

In [31]:
def estilo(fig):
    fig.update_layout(
        title_font=dict(size=24, family="Arial", color="black"),
        xaxis_tickfont=dict(size=12),
        yaxis_tickfont=dict(size=12),
        plot_bgcolor="white",
        paper_bgcolor="white",
        showlegend=False,
        margin=dict(t=80, l=40, r=20, b=120),
    )
    
    return fig

In [32]:
df = pl.scan_csv(
    get_file_path('cleaned', 'exportaciones'),
    low_memory=True
).collect()

## Análisis específicos

### Serie de tiempo mensual

In [33]:
serie_mensual = df.group_by("FECH").agg(pl.col("FOBDOL").sum()).sort("FECH")

In [34]:
fig = px.line(serie_mensual,
              x='FECH',
              y='FOBDOL',
              title='Serie de tiempo mensual',
              labels={"FECH": "AÑO - MES", "FOBDOL": "FOB Dólares"},
              )
fig = estilo(fig)
fig.update_layout(xaxis_tickangle=45)
fig.update_xaxes(
    tickformat="%Y-%m",
    dtick="M12"
)
fig.show()

###### *__Finalidad:__ Mostrar la evolución detallada del valor mensual de las exportaciones a lo largo del tiempo, permitiendo identificar picos, caídas y cambios mes a mes.*

### Distribución Mensual de los valores FOBDOL por Año

In [35]:
serie_heatmap = (
    df.with_columns([
        pl.col("AÑO"),
        pl.col("MES")
    ])
    .group_by(["AÑO", "MES"])
    .agg(pl.col("FOBDOL").sum().alias("FOBDOL"))
    .sort(["AÑO", "MES"])
)

In [36]:
serie_heatmap = serie_heatmap.to_pandas()

In [37]:
pivot_table = serie_heatmap.pivot(index="AÑO", columns="MES", values="FOBDOL")

In [38]:
serie_heatmap["FOBDOL_LOG"] = serie_heatmap["FOBDOL"].apply(lambda x: np.log10(x + 1))

pivot_log = serie_heatmap.pivot(index="AÑO", columns="MES", values="FOBDOL_LOG")
pivot_raw = serie_heatmap.pivot(index="AÑO", columns="MES", values="FOBDOL")

fig = go.Figure(data=go.Heatmap(
    z=pivot_log.values,
    x=[calendar.month_abbr[m] for m in pivot_log.columns],
    y=pivot_log.index,
    colorscale="YlGnBu",
    colorbar=dict(
        title="Fobdol",
        tickformat=".2f",
        ticks="outside",
        tickmode="auto"
    ),
    hovertemplate=
        "<b>Año:</b> %{y}<br>" +
        "<b>Mes:</b> %{x}<br>" +
        "<b>FOBDOL:</b> %{z:$,.2f}<extra></extra>"
))

fig.update_layout(
    title="FOB Dólares por Mes y Año (escala log)",
    xaxis_title="Mes",
    yaxis_title="Año",
    xaxis_tickfont=dict(size=12),
    yaxis_tickfont=dict(size=12),
    xaxis=dict(type="category"),
    yaxis=dict(type="category"),
    title_font=dict(size=24, family="Arial", color="black"),
    plot_bgcolor="white",
    paper_bgcolor="white"
)

fig.show()

###### *__Finalidad:__ Este heatmap muestra la suma de los valores FOB por mes y año. Permite identificar patrones estacionales, variaciones interanuales y meses con mayores niveles de exportación.*

### Crecimiento anual (%) del valor FOB de las exportaciones

In [39]:
crecimiento = (
    df.group_by("AÑO")
      .agg(pl.col("FOBDOL").sum())
      .sort("AÑO")
      .with_columns(
          (pl.col("FOBDOL").pct_change() * 100)
          .fill_null(0.0)
          .round(2)
          .alias("crecimiento_anual")
      )
)

In [40]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=crecimiento["AÑO"],
    y=crecimiento["crecimiento_anual"],
    text=crecimiento["crecimiento_anual"],
    textposition="outside",
    marker_color="steelblue",
))

fig.update_layout(
    title="Crecimiento anual del valor FOB de las exportaciones",
    title_font=dict(size=24, family="Arial", color="black"),
    xaxis_title="Año",
    yaxis_title="Crecimiento (%)",
    xaxis_tickfont=dict(size=12),
    yaxis_tickfont=dict(size=12),
    yaxis_tickformat=".0f",
    xaxis_tickangle=45,
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    plot_bgcolor="white",
    paper_bgcolor="white",
    bargap=0.2,
    showlegend=False,
    margin=dict(t=80, l=40, r=20, b=40),
)
fig.update_xaxes(
    tickformat="%Y",
    dtick="Y12"
)
fig.show()

###### *__Finalidad:__ Medir la variación porcentual del valor exportado entre un año y el siguiente, permitiendo evaluar ritmos de crecimiento o disminución.*

### Tendencia de largo plazo del valor FOB de las exportaciones

In [ ]:
anual = df.group_by("AÑO").agg(pl.col("FOBDOL").sum()).sort("AÑO")
x = anual.select("AÑO").to_numpy().flatten()
y = anual.select("FOBDOL").to_numpy().flatten()

slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
print(f"Tendencia anual: pendiente={slope:.2f}, R²={r_value**2:.3f}")

Tendencia anual: pendiente=1030418761.96, R²=0.255


In [42]:
y_pred = intercept + slope * x

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode="markers",
    name="Exportaciones anuales",
    marker=dict(color="steelblue", size=8)
))

fig.add_trace(go.Scatter(
    x=x,
    y=y_pred,
    mode="lines",
    name="Tendencia lineal",
    line=dict(color="red", dash="dash")
))

fig.update_layout(
    title="Tendencia de largo plazo del valor FOB de las exportaciones",
    xaxis_title="Año",
    yaxis_title="FOB Dólares",
    title_font=dict(size=24, family="Arial", color="black"),
    xaxis_tickfont=dict(size=12),
    yaxis_tickfont=dict(size=12),
    plot_bgcolor="white",
    showlegend=True
)
fig.show()

###### *__Finalidad:__ Identificar la tendencia general de las exportaciones a lo largo del tiempo mediante una regresión lineal, lo que permite observar si existe un crecimiento o disminución sostenida en el valor FOB anual y qué tan bien se ajusta esta tendencia a los datos históricos.*